In [ ]:
import torch
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader, Dataset 
import numpy as np

In [ ]:
 #Compose에 필요한 전처리 방법 대입
transf = tr.Compose([tr.Resize(8),tr.ToTensor()]) #Transforms on PIL Image

In [ ]:
#1. 파이토치 제공 데이터 사용
train = torchvision.datasets.CIFAR10(root='./data',train=True,download=True, transform=transf)
test = torchvision.datasets.CIFAR10(root='./data',train=False,download=True, transform=transf)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
train[0][0].size() #채널3개에 8*8이미지

torch.Size([3, 8, 8])

In [ ]:
trainloader = DataLoader(train,batch_size=50,shuffle=True,num_workers=2) #배치 사이즈 정하기
testloader = DataLoader(test,batch_size=50,shuffle=True,num_workers=2)

In [ ]:
len(trainloader) #배치 사이즈 50개인 배치가 1000개 있음

1000

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [ ]:
images.size() #배치 사이즈 50, 채널 3, 너비 8, 높이 8

torch.Size([50, 3, 8, 8])

In [ ]:
#2. 같은 클래스 별 폴더 이미지 데이터 이용

In [ ]:
# ./class/tiger ./class/lion
transf = tr.Compose([tr.Resize(16),tr.ToTensor()])
trainset = torchvision.datasets.ImageFolder(root='./class',transform=transf) #ImageFolder: 각각의 다른 폴더에 대해서 다른 레이블을 달아줌
trainloader = DataLoader(trainset, batch_size=1,shuffle=False,num_works=2)
print(len(trainloader))

FileNotFoundError: ignored

In [ ]:
#3. 개인 데이터 사용
#import preprocessing

train_images = np.random.randint(256,size=(20,32,32,3)) #20*20 3채널 이미지가 20개
train_labels = np.random.randint(2,size=(20,1)) #레이블 20개

#preprocessing...
#train_images, train_labels = preprocessing(train_images, train_labels)

print(train_images.shape, train_labels.shape)

(20, 32, 32, 3) (20, 1)


In [ ]:
#데이터셋을 상속 받을 클래스를 지정
class TensorData(Dataset):

  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data) #Float/Long: 숫자 속성
    self.x_data = self.x_data.permute(0,3,1,2) #이미지 개수, 채널 수, 이미지 너비, 높이
    self.y_data =  torch.LongTensor(y_data)
    self.len = self.y_data.shape[0]

  def __getitem__(self,index):

    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len 

In [ ]:
train_data = TensorData(train_images, train_labels)
train_loader = DataLoader(train_data,batch_size=10,shuffle=True)

In [ ]:
train_data[0][0].size()

torch.Size([3, 32, 32])

In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

In [ ]:
images.size()

torch.Size([10, 3, 32, 32])

In [ ]:
### from torch.utils.data import Dataset
class MyDataset(Dataset):

  def __init__(self):
  
  def __getitem__(self,index):

  def __len__(self):

In [ ]:
class MyDataset(Dataset):

  def __init__(self,x_data,y_data,transform=None):
    self.x_data = x_data
    self.y_data = y_data
    self.transform = transform
    self.len = len(y_data)

  def __getitem__(self,index):
    sample = self.x_data[index], self.y_data[index]

    if self.transform:
      sample = self.transform(sample) #전처리

    return sample

  def __len__(self):
    return self.len 

class ToTensor:

  def __call__(self, sample):
    inputs, labels = sample
    inputs = torch.FloatTensor(inputs)
    inputs = inputs.permute(2,0,1)
    return inputs, torch.LongTensor(labels)

class LinearTensor:
   
   def __init__(self, slope=1, bias=0):
     self.slope = slope
     self.bias = bias
   def __call__(self, sample):
      inputs, lables = sample
      inputs = self.slope*inputs + self.bias
      return inputs, labels

In [ ]:
trans = tr.Compose([ToTensor(),LinearTensor(2,5)])
ds1 = MyDataset(train_images, train_labels, transform=trans)
train_loader1 = DataLoader(ds1,batch_size=10, shuffle=True)

In [ ]:
first_data = ds1[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [ ]:
dataiter1 = iter(train_loader1)
images1, labels1 = dataiter1.next()

In [ ]:
images1

tensor([[[[487., 435., 319.,  ...,  81., 471.,  81.],
          [303.,  53., 251.,  ..., 335., 279., 421.],
          [491., 297., 505.,  ..., 453., 515., 203.],
          ...,
          [471.,  55., 513.,  ..., 505., 383., 289.],
          [167., 413., 281.,  ...,  25.,  87., 191.],
          [219., 281., 363.,  ..., 249., 187., 415.]],

         [[331., 131., 293.,  ..., 225., 485., 135.],
          [389., 281., 389.,  ..., 499., 251., 231.],
          [ 49.,  15., 505.,  ..., 281., 163.,  45.],
          ...,
          [277., 153., 127.,  ..., 391., 169., 139.],
          [505., 337.,  55.,  ...,  51.,  93., 309.],
          [389., 149., 505.,  ..., 153., 403., 369.]],

         [[317., 311., 489.,  ..., 447., 433., 225.],
          [209.,  53., 289.,  ...,  49., 109., 111.],
          [351., 511., 265.,  ..., 313., 425., 331.],
          ...,
          [191., 263., 167.,  ..., 365.,  49., 205.],
          [ 31., 477., 407.,  ..., 175., 257., 253.],
          [ 81., 471., 347.,  ...

In [ ]:
class MyDataset(Dataset):

  def __init__(self,x_data,y_data,transform=None):
    self.x_data = x_data
    self.y_data = y_data
    self.transform = transform
    self.len = len(y_data)

  def __getitem__(self,index):
    sample = self.x_data[index], self.y_data[index]

    if self.transform:
      sample = self.transform(sample) #전처리

    return sample

  def __len__(self):
    return self.len 

class MyTransform:

  def __call__(self, sample):
    inputs, lables = sample
    inputs = torch.FloatTensor(inputs)
    inputs = inputs.permute(2,0,1)
    labels = torch.FloatTensor(lables)

    transf = tr.Compose([tr.ToPILImage(), tr.Resize(128), tr.ToTensor(),tr.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
    final_output = transf(inputs)

    return final_output, lables

In [ ]:
ds2 = MyDataset(train_images,train_labels, transform=MyTransform())
train_loader2 = DataLoader(ds2,batch_size=10,shuffle=True)

In [ ]:
first_data = ds2[0]
features, lables = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [ ]:
dataiter2 = iter(train_loader2)
images2, labels2 = dataiter2.next()

In [ ]:
images2.size()

torch.Size([10, 3, 128, 128])

In [ ]:
images2

tensor([[[[-0.8824, -0.8824, -0.8588,  ...,  0.5216,  0.7098,  0.7098],
          [-0.8824, -0.8824, -0.8588,  ...,  0.5216,  0.7098,  0.7098],
          [-0.7882, -0.7882, -0.7569,  ...,  0.3882,  0.5451,  0.5451],
          ...,
          [ 0.1922,  0.1922,  0.1529,  ..., -0.3882, -0.4902, -0.4902],
          [ 0.1686,  0.1686,  0.1373,  ..., -0.4902, -0.6000, -0.6000],
          [ 0.1686,  0.1686,  0.1373,  ..., -0.4902, -0.6000, -0.6000]],

         [[-0.2706, -0.2706, -0.1686,  ...,  0.3255,  0.4980,  0.4980],
          [-0.2706, -0.2706, -0.1686,  ...,  0.3255,  0.4980,  0.4980],
          [-0.3020, -0.3020, -0.2000,  ...,  0.3020,  0.4510,  0.4510],
          ...,
          [-0.5529, -0.5529, -0.4431,  ..., -0.3882, -0.3882, -0.3882],
          [-0.4980, -0.4980, -0.3804,  ..., -0.4353, -0.4196, -0.4196],
          [-0.4980, -0.4980, -0.3804,  ..., -0.4353, -0.4196, -0.4196]],

         [[-0.2157, -0.2157, -0.2157,  ...,  0.0431,  0.1451,  0.1451],
          [-0.2157, -0.2157, -